In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from keras.models import Model
from keras.layers import Dense, Input, add, BatchNormalization, concatenate, Embedding, Flatten 
from tensorflow.keras.optimizers import Nadam
from keras.layers.core import Dropout, Activation
from keras.initializers import RandomNormal, Ones

import tensorflow as tf

import seaborn as sns

import random
from numpy import array
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Function PD: Calculate Poisson Deviance
def PD_function(pred, obs):
    PD = 200*( sum(pred) - sum(obs) + sum( np.log( (obs/pred)**(obs) ) ) )
    return PD/len(pred)
    
# evaluate a single mlp model
def evaluate_model(trainX, trainy, train_expo, trainbrand, trainregion,
                   activ_fonction_0, activ_fonction_1, activ_fonction_2,
                   nb_neurones_0, nb_neurones_1, nb_neurones_2, 
                   drop_out_1, drop_out_2
                  ):
    
    first_input = Input(shape=(7))
    
    brand_input = Input(shape=(1))
    region_input = Input(shape=(1))    
    
    
    Embedding_brand = Flatten()(Embedding(input_dim = 11, output_dim = 2)(brand_input))
    Embedding_region = Flatten()(Embedding(input_dim = 22, output_dim = 2)(region_input))
    
    
    merge_one = tf.keras.layers.Concatenate()([Embedding_brand, Embedding_region, first_input])
    
    #first_dense_0 = Dense(nb_neurones_0, kernel_initializer=RandomNormal(mean=0.0, stddev=1, seed=10))(merge_one)
    #act_dense_0 = Activation(activ_fonction_0)(first_dense_0)
    
    first_dense_1 = Dense(nb_neurones_1)(merge_one)
    act_dense_1 = Activation(activ_fonction_1)(first_dense_1)
    
    
    first_dense_2 = Dense(nb_neurones_2)(act_dense_1)
    act_dense_2 = Activation(activ_fonction_2)(first_dense_2)

    first_dense_3 = Dense(1, 
                          bias_initializer=tf.keras.initializers.Constant(value=-1.7942), 
                          #kernel_initializer=tf.keras.initializers.Ones()
                         )(act_dense_2)
    act_dense_3 = Activation('linear')(first_dense_3)

    second_input = Input(shape=(1, ))

    second_dense_1 = Dense(1, 
                           bias_initializer=tf.keras.initializers.Constant(value=-1.66), 
                           kernel_initializer=tf.keras.initializers.Constant(value=0.43), 
                           trainable=True)(add([act_dense_3, second_input])) 
    final_output = Activation('exponential')(second_dense_1) 

    model_ = Model(inputs=[first_input, second_input, brand_input, region_input], outputs=final_output)

    model_.compile(optimizer=Nadam(), loss='poisson')
    
    model_.fit([trainX, train_expo, trainbrand, trainregion], trainy, epochs=450, verbose=1,
                   batch_size=10000
                  )

    return model_, 0

In [3]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, X_test, expo_test, brand_test, region_test):
    
    # make predictions
    yhats = [model.predict([X_test, expo_test, brand_test, region_test]) for model in members]
    yhats = array(yhats)
    
    result = np.mean(yhats, axis=0).flatten() 
    
    return result


# evaluate a specific number of members in an ensemble
def evaluate_n_members_bis(members, testX, testexpo, testbrand, testregion):
    
    # select a subset of members
    subset = members[:len(members)]
    
    # make prediction
    yhat = ensemble_predictions(subset, testX, testexpo, testbrand, testregion)
    
    # calculate accuracy 
    return yhat


def Ensemble_fit(Xtrain_, ytrain_, expotrain_, brandtrain_, regiontrain_,
                 samples=30, rate_OOB=0.2, type_ = "Bagging", ver_freq_train=True):
    
    n_splits = samples
    scores_, members_ = list(), list()
   
    activ_fonction_0, activ_fonction_1, activ_fonction_2 = random.sample(["tanh"], 1), random.sample(["tanh"], 1), random.sample(["tanh"], 1)
    nb_neurones_0, nb_neurones_1, nb_neurones_2 = random.sample([20], 1), random.sample([15], 1), random.sample([10], 1)
    drop_out_1, drop_out_2 = random.sample([0.02, 0.04, 0.06, 0.08], 1), random.sample([0.02, 0.04, 0.06, 0.08], 1)
        
        # evaluate model
    model, test_acc = evaluate_model(Xtrain_, ytrain_, expotrain_, brandtrain_, regiontrain_, 
                                     activ_fonction_0[0], activ_fonction_1[0], activ_fonction_2[0], 
                                     nb_neurones_0[0], nb_neurones_1[0], nb_neurones_2[0], 
                                     drop_out_1[0], drop_out_2[0])
        
    print('> OOB > %.3f' % test_acc)
    scores_.append(test_acc)
    members_.append(model)   
    
    return members_, scores_

, 81, 37 ,71, 28, 70, 54, 59, 17, 79

, 54, 42, 88, 29

In [4]:
df_poisson = pd.DataFrame()

table_name = ["deviance_train", "deviance_test", "diff_train", "diff_test"]

for p in [62, 54, 42, 88, 29]:
    
    print(p)
    
    data_learn = pd.read_csv(r'C:\Users\KRASNIQ\Documents\These_1.1\data\data_learn_'+str(62)+'.csv')
    data_test = pd.read_csv(r'C:\Users\KRASNIQ\Documents\These_1.1\data\data_test_'+str(62)+'.csv')
    
    nex_data = pd.concat([data_learn, data_test])
    
    data_learn_, data_test_ = train_test_split(nex_data, stratify=nex_data['ClaimNb'], test_size=0.1, random_state=p)
    
    data_learn_['Exposure'] = data_learn_['Exposure'].clip(upper=1)
    data_test_['Exposure'] = data_test_['Exposure'].clip(upper=1)
    
    data_learn_['Area'] = data_learn_['Area'].map({"A": 1, "B": 2,"C": 3,"D": 4,"E": 5,"F": 6,})
    data_test_['Area'] = data_test_['Area'].map({"A": 1, "B": 2,"C": 3,"D": 4,"E": 5,"F": 6,})
    
    data_learn_ = data_learn_.drop(["Area_class_1","Area_class_2","Area_class_3","Area_class_4",'ClaimAmount', 'DrivAge', 'BonusMalus',
                                    "VehBrand_class_1", "VehBrand_class_2", "VehBrand_class_3",
                                    'Density',"Region_class_1", "Region_class_2", "Region_class_3", "Region_class_4" , 
                                    'VehAge', 'VehGas', "VehGas_class_2"], axis=1)

    data_test_ = data_test_.drop(["Area_class_1","Area_class_2","Area_class_3","Area_class_4",'ClaimAmount', 'DrivAge', 'BonusMalus',
                                  "VehBrand_class_1", "VehBrand_class_2", "VehBrand_class_3",
                                  'Density',"Region_class_1", "Region_class_2", "Region_class_3", "Region_class_4" , 
                                  'VehAge', 'VehGas', "VehGas_class_2"], axis=1)
    
    # train test split 
    X_train__, X_valid__ = train_test_split(data_learn_, stratify=data_learn_['ClaimNb'], test_size=0.15, random_state=p)
    
    
    #################### label encoding
    
    
    le_region = preprocessing.LabelEncoder()
    le_region.fit(X_train__['Region'])

    le_brand = preprocessing.LabelEncoder()
    le_brand.fit(X_train__['VehBrand'])
    
    y_train = np.array(X_train__.filter(['ClaimNb']))
    log_exp_train= np.array(np.log(X_train__.filter(['Exposure'])))
    region_train = np.array(le_region.transform(X_train__['Region']))
    brand_train = np.array(le_brand.transform(X_train__['VehBrand']))
    x_train = X_train__.drop(columns=['ClaimNb', 'Exposure', 'VehBrand', 'Region'])

    y_valid = np.array(X_valid__.filter(['ClaimNb']))
    log_exp_valid= np.array(np.log(X_valid__.filter(['Exposure'])))
    region_valid = np.array(le_region.transform(X_valid__['Region']))
    brand_valid = np.array(le_brand.transform(X_valid__['VehBrand']))
    x_valid = X_valid__.drop(columns=['ClaimNb', 'Exposure', 'VehBrand', 'Region'])
    
    #valid
    y_test = np.array(data_test_.filter(['ClaimNb']))
    log_exp_test = np.array(np.log(data_test_.filter(['Exposure'])))
    region_test = np.array(le_region.transform(data_test_['Region']))
    brand_test = np.array(le_brand.transform(data_test_['VehBrand']))
    x_test = data_test_.drop(columns=['ClaimNb', 'Exposure', 'VehBrand', 'Region'])
    
    #feature eng
    cs = MinMaxScaler()
    X_train = np.array(2*((x_train - x_train.min())/(x_train.max()-x_train.min())) - 1)
    X_valid = np.array(2*((x_valid - x_train.min())/(x_train.max()-x_train.min())) - 1)
    X_test = np.array(2*((x_test - x_train.min())/(x_train.max()-x_train.min())) - 1)
    
    
    members, scores = Ensemble_fit(X_train, y_train, log_exp_train, brand_train, region_train,
                                   samples=1, rate_OOB=0, type_ = "Bagging",
                                   ver_freq_train=False)

    
    predictions_test = evaluate_n_members_bis(members, X_test, log_exp_test, brand_test, region_test)
    predictions_train = evaluate_n_members_bis(members, X_train, log_exp_train, brand_train, region_train)

    #deviance
    table_modal= [np.round(PD_function(predictions_train, y_train.flatten()),6),
                  np.round(PD_function(predictions_test, y_test.flatten()),6),
                  (sum(predictions_train) - sum(y_train.flatten()))/sum(y_train.flatten())*100,
                  (sum(predictions_test) -sum(y_test.flatten()))/sum(y_test.flatten())*100]
    
    table_name = ["deviance_train", "deviance_test", "diff_train", "diff_test"]

    cnt = dict()
    k=0
    for word in table_name:
        cnt[word] = table_modal[k]
        k = k+1
    df_poisson = df_poisson.append(cnt, ignore_index=True)

62
Epoch 1/450
52/52 [==============================] - 1s 5ms/step - loss: 0.2066
Epoch 2/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2053
Epoch 3/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2044
Epoch 4/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2037
Epoch 5/450
52/52 [==============================] - 0s 6ms/step - loss: 0.2032
Epoch 6/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2030
Epoch 7/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2029
Epoch 8/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2029
Epoch 9/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2029
Epoch 10/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2028
Epoch 11/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2028
Epoch 12/450
52/52 [==============================] - 0s 5ms/step - loss: 0.2028
Epoch 13/450
52/52 [==============

52/52 [==============================] - 0s 5ms/step - loss: 0.2000
Epoch 103/450
52/52 [==============================] - 0s 8ms/step - loss: 0.2000
Epoch 104/450
52/52 [==============================] - 0s 6ms/step - loss: 0.2000
Epoch 105/450
52/52 [==============================] - 0s 7ms/step - loss: 0.2000
Epoch 106/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1999
Epoch 107/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1999
Epoch 108/450
52/52 [==============================] - 0s 4ms/step - loss: 0.1999
Epoch 109/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1999
Epoch 110/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1999
Epoch 111/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1998
Epoch 112/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1998
Epoch 113/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1998
Epoch 114/450
52/52 [=========

52/52 [==============================] - 0s 5ms/step - loss: 0.1985
Epoch 203/450
52/52 [==============================] - 0s 8ms/step - loss: 0.1984
Epoch 204/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1984
Epoch 205/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1984
Epoch 206/450
52/52 [==============================] - 0s 7ms/step - loss: 0.1984
Epoch 207/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1984
Epoch 208/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1984
Epoch 209/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1984
Epoch 210/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1984
Epoch 211/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1984
Epoch 212/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1984
Epoch 213/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1984
Epoch 214/450
52/52 [=========

52/52 [==============================] - 0s 6ms/step - loss: 0.1979
Epoch 303/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1979
Epoch 304/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1979
Epoch 305/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1979
Epoch 306/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1979
Epoch 307/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1979
Epoch 308/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1979
Epoch 309/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1979
Epoch 310/450
52/52 [==============================] - 0s 8ms/step - loss: 0.1979
Epoch 311/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1979
Epoch 312/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1978
Epoch 313/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1979
Epoch 314/450
52/52 [=========

52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 403/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 404/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 405/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 406/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1976
Epoch 407/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 408/450
52/52 [==============================] - 0s 4ms/step - loss: 0.1976
Epoch 409/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 410/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 411/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1976
Epoch 412/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 413/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1976
Epoch 414/450
52/52 [=========

...

52/52 [==============================] - 0s 5ms/step - loss: 0.1999
Epoch 103/450
52/52 [==============================] - 0s 8ms/step - loss: 0.1999
Epoch 104/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1999
Epoch 105/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1998
Epoch 106/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1998
Epoch 107/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1998
Epoch 108/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1998
Epoch 109/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1998
Epoch 110/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1997
Epoch 111/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1997
Epoch 112/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1997
Epoch 113/450
52/52 [==============================] - 0s 7ms/step - loss: 0.1997
Epoch 114/450
52/52 [=========

52/52 [==============================] - 0s 5ms/step - loss: 0.1982
Epoch 203/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1982
Epoch 204/450
52/52 [==============================] - 0s 7ms/step - loss: 0.1982
Epoch 205/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1982
Epoch 206/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1982
Epoch 207/450
52/52 [==============================] - 0s 7ms/step - loss: 0.1982
Epoch 208/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1982
Epoch 209/450
52/52 [==============================] - 0s 7ms/step - loss: 0.1982
Epoch 210/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1981
Epoch 211/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1982
Epoch 212/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1981
Epoch 213/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1981
Epoch 214/450
52/52 [=========

52/52 [==============================] - 0s 6ms/step - loss: 0.1977
Epoch 303/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1977
Epoch 304/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 305/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 306/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 307/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 308/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 309/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1977
Epoch 310/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 311/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 312/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 313/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1977
Epoch 314/450
52/52 [=========

52/52 [==============================] - 0s 6ms/step - loss: 0.1975
Epoch 403/450
52/52 [==============================] - 0s 7ms/step - loss: 0.1975
Epoch 404/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1975
Epoch 405/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1975
Epoch 406/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1975
Epoch 407/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1975
Epoch 408/450
52/52 [==============================] - 0s 8ms/step - loss: 0.1975
Epoch 409/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1975
Epoch 410/450
52/52 [==============================] - 0s 6ms/step - loss: 0.1975
Epoch 411/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1975
Epoch 412/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1975
Epoch 413/450
52/52 [==============================] - 0s 5ms/step - loss: 0.1975
Epoch 414/450
52/52 [=========

C:\Users\KRASNIQ\AppData\Local\Temp\ipykernel_23220\3487817685.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_poisson = df_poisson.append(cnt, ignore_index=True)


for layer in members[0].layers:
    weights = layer.get_weights() # list of numpy arrays
    print(weights)

In [5]:
df_poisson['model']=["cv-bagging-1"]*len(df_poisson)

In [6]:
df_poisson

,deviance_train,deviance_test,diff_train,diff_test,model
0,29.685155,29.872470,3.166761,3.223092,cv-bagging-1
1,29.641166,29.665972,0.174574,0.281067,cv-bagging-1
2,29.685336,29.734628,-2.287122,-2.415117,cv-bagging-1
3,29.658156,29.841751,-3.897947,-4.155277,cv-bagging-1
4,29.674656,29.855803,-2.822441,-2.586099,cv-bagging-1


In [7]:
df_poisson["deviance_train"].mean(), df_poisson["deviance_test"].mean(), df_poisson["diff_train"].mean(), df_poisson["diff_test"].mean()

(29.6688938, 29.794124800000002, -1.1332348687359441, -1.1304667890137725)

In [8]:
df_poisson["deviance_train"].std(), df_poisson["deviance_test"].std(), df_poisson["diff_train"].std(), df_poisson["diff_test"].std()

(0.01905325928023917,
 0.08968477435272941,
 2.8297012915547044,
 2.909663737165378)

In [9]:
data_learn.columns

Index(['ClaimNb', 'Exposure', 'Area', 'VehPower', 'VehAge', 'DrivAge',
       'BonusMalus', 'VehBrand', 'VehGas', 'Density', 'Region', 'ClaimAmount',
       'BonusMalus_capped', 'VehAge_capped', 'DrivAge_capped', 'LogDensity',
       'VehBrand_class_1', 'VehBrand_class_2', 'VehBrand_class_3',
       'Area_class_1', 'Area_class_2', 'Area_class_3', 'Area_class_4',
       'Region_class_1', 'Region_class_2', 'Region_class_3', 'Region_class_4',
       'VehGas_class_1', 'VehGas_class_2'],
      dtype='object')

In [10]:
df_matrix_train_ = 0
df_matrix_valid_ = 0

cf_matrix_train = confusion_matrix(y_train, np.round(predictions_train).astype(int))
print(cf_matrix_train)

if cf_matrix_train.shape[0] == 4:
    cf_matrix_train = np.append(cf_matrix_train, [[0]*cf_matrix_train.shape[1]], axis=0)

if cf_matrix_train.shape[1] == 2:
    cf_matrix_train = np.append(cf_matrix_train, [[0]*cf_matrix_train.shape[0]], axis=1)
if cf_matrix_train.shape[1] == 3:
    cf_matrix_train = np.append(cf_matrix_train, [[0]*cf_matrix_train.shape[0]], axis=1)
if cf_matrix_train.shape[1] == 4:
    cf_matrix_train = np.append(cf_matrix_train, [[0]*cf_matrix_train.shape[0]], axis=1)
    
df_matrix_train_ = df_matrix_train_ + cf_matrix_train
    
cf_matrix_valid = confusion_matrix(y_test, np.round(predictions_test).astype(int))
print(cf_matrix_valid)
    
if cf_matrix_valid.shape[0] == 4:
    cf_matrix_valid = np.append(cf_matrix_valid, [[0]*cf_matrix_valid.shape[1]], axis=0)

if cf_matrix_valid.shape[1] == 2:
    cf_matrix_valid = np.append(cf_matrix_valid, np.zeros((cf_matrix_valid.shape[0],1)), axis=1)
if cf_matrix_valid.shape[1] == 3:
    cf_matrix_valid = np.append(cf_matrix_valid, np.zeros((cf_matrix_valid.shape[0],1)), axis=1)
if cf_matrix_valid.shape[1] == 4:
    cf_matrix_valid = np.append(cf_matrix_valid, np.zeros((cf_matrix_valid.shape[0],1)), axis=1)
    
df_matrix_valid_ = df_matrix_valid_ + cf_matrix_valid

[[492590     16      0      0      0]
 [ 24604     12      0      0      0]
 [  1364      1      0      0      0]
 [    63      0      0      0      0]
 [    12      0      0      0      0]]
[[64391     2     0     0     0]
 [ 3217     1     0     0     0]
 [  177     1     0     0     0]
 [    8     0     0     0     0]
 [    2     0     0     0     0]]


In [11]:
y_train.mean(), y_test.mean()

(0.05318106975255561, 0.053186625171462706)